In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchWindowException, NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager



import pandas as pd
import time
#ser = Service(r"chromedriver.exe")
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

In [20]:
#driver = webdriver.Chrome(service=ser, options=options)
#driver = webdriver.Chrome(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

[WDM] - Downloading:  98%|█████████▊| 6.64M/6.78M [00:00<00:00, 26.1MB/s]

In [21]:
def scrape_from_individual_job_url(link: str):
    time.sleep(5)
    driver.get(link)
    
    job_title = driver.find_element(By.CLASS_NAME, "icl-u-xs-mb--xs.icl-u-xs-mt--none.jobsearch-JobInfoHeader-title").text
    
    data = driver.find_element(By.CLASS_NAME, "icl-u-xs-mt--xs.icl-u-textColor--secondary.jobsearch-JobInfoHeader-subtitle.jobsearch-DesktopStickyContainer-subtitle")
    
    data_list = data.text.split('\n')
    company_name = data_list[0]
    company_location = data_list[-1]
    
    job_description = driver.find_element(By.CLASS_NAME, "jobsearch-jobDescriptionText").text
    #job_activity = driver.find_element(By.CLASS_NAME, "jobsearch-HiringInsights-entry--text").text
    
    job_activity_strings = driver.find_element(By.CLASS_NAME, "css-q7fux.eu4oa1w0").text
    job_activity = job_activity_strings.split("\n")[2]
    
    try:
        benefits = driver.find_elements(By.CLASS_NAME, "css-tvvxwd.ecydgvn1")
        benefits_list = [benefit.text for benefit in benefits]    
    except:
        benefits_list = None
        
    return job_title, company_name, company_location, job_description, job_activity, benefits_list

In [22]:
def scrape_all_job_data(job_title_to_search: str, city: str, age: int):    
    job_title_to_search = job_title_to_search.replace(" ", "+")
    url_template = "https://www.indeed.com/jobs?q={}&l={}&fromage={}".format(job_title_to_search, city,age)

    driver.get(url_template)
    original_page = driver.current_url
    original_window = driver.current_window_handle

    href_elements = driver.find_elements(By.CLASS_NAME, "jcs-JobTitle.css-jspxzf")
    links = [elem.get_attribute('href') for elem in href_elements]
    
    job_titles, company_names, company_locations, job_descriptions, job_activities, benefits_lists = [], [], [], [], [], []
    
    driver.switch_to.new_window()
    tab_that_goes_to_individual_jobs = driver.current_window_handle
    
    for link in links:
        try:
            job_title, company_name, company_location, job_description, job_activity, benefits_list = scrape_from_individual_job_url(link)

            job_titles.append(job_title)
            company_names.append(company_name)
            company_locations.append(company_locations)
            job_descriptions.append(job_description)
            job_activities.append(job_activity)
            benefits_lists.append(benefits_list)
            
        except (NoSuchWindowException):
            print("No Windows")
            pass
        except NoSuchElementException as e:
            print(e)
            pass
        
    driver.switch_to.window(original_window)
    
    driver.find_element(By.XPATH, '//a[@aria-label="Next Page"]').click()
            
    scraped_data_df = pd.DataFrame(data = {"Job Title": job_titles, 
                                           "Company Name": company_names, 
                                           "Company Location": company_locations,
                                           "Job Description": job_descriptions,
                                           "Benefits": benefits_lists,
                                           "Job Posted Date": job_activities})
    
    
    return scraped_data_df

In [23]:
df = scrape_all_job_data("Data Scientist", "Philadelphia",15)

[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [00:20<00:00, 26.1MB/s]

In [24]:
df

,Job Title,Company Name,Company Location,Job Description,Benefits,Job Posted Date
0,"Data Scientist 2, Enterprise Data Analytics & ...",Comcast,"[[[[[...], [...], [...], [...], [...], [...], ...",Comcast brings together the best in media and ...,[],Posted 1 day ago
1,Data Scientist,Liberty Personnel Services,"[[[[[...], [...], [...], [...], [...], [...], ...",My client hired me to find for them a data sci...,[],Posted 11 days ago
2,Software Engineer - LabVIEW - AI - Machine Vis...,Sciotex,"[[[[[...], [...], [...], [...], [...], [...], ...",Sciotex is looking for skilled software develo...,"[401(k), Flexible schedule, Health insurance]",Job activity
3,Data Quality Assurance - Food Science,"Task Management, Inc","[[[[[...], [...], [...], [...], [...], [...], ...",GRADUATES WITH SCIENCE DEGREE– Contract role -...,[Health insurance],Urgently hiring
4,Data Scientist (Mid Level),"Khoj Information Technology, Inc.","[[[[[...], [...], [...], [...], [...], [...], ...",HYBRID/REMOTE - STRONG PREFERENCE for HYBRID -...,[],Job activity
5,Data Scientist,Harris Blitzer Sports & Entertainment,"[[[[[...], [...], [...], [...], [...], [...], ...",The Data Scientist will work in the Research a...,"[401(k), 401(k) matching, Dental insurance, Fl...",Posted 13 days ago
6,Applied AI/ML Data Scientist - Card Portfolio ...,"JPMorgan Chase Bank, N.A.","[[[[[...], [...], [...], [...], [...], [...], ...",Job Title\nApplied AI/ML Data Scientist\n\nSho...,"[Health insurance, Retirement plan, Tuition re...",Posted 2 days ago
7,Data Scientist,University of Pennsylvania,"[[[[[...], [...], [...], [...], [...], [...], ...",University Overview\n\nThe University of Penns...,"[Adoption assistance, Continuing education cre...",Posted 5 days ago
8,Data Scientist II,Burlington Stores,"[[[[[...], [...], [...], [...], [...], [...], ...",LOCATION 4287 Route 130 S Edgewater Park NJ US...,"[401(k), Dental insurance, Disability insuranc...",Posted 9 days ago
9,Scientist NAFS R&D Product Data Management (PDM),Campbell Soup Company,"[[[[[...], [...], [...], [...], [...], [...], ...",Imagine...working for a company that knows tha...,[],Posted 13 days ago


In [ ]:
df = scrape_all_job_data("Software Engineer", "Philadelphia")

In [ ]:
df.head()

In [ ]:
# Page 1 start=0, Page2 start=10 ... Page 15 start=140

current_page = print(driver.current_url)

In [ ]:
# Number of jobs
<div class="jobsearch-JobCountAndSortPane-jobCount">

In [ ]:
job_title = driver.find_element(By.CLASS_NAME, "icl-u-xs-mb--xs.icl-u-xs-mt--none.jobsearch-JobInfoHeader-title")
print(job_title.text)

In [ ]:
driver.get('https://www.indeed.com/viewjob?jk=ecbcf343b3e3b8ce&tk=1gfjvhrjeg0n6802&from=serp&vjs=3&advn=4753071484186304&adid=359869966&ad=-6NYlbfkN0Cj-KmZPsf9w80C8b1WzNVrlanjD2SXJjxuCbUWHsXPZkFBy4Qr63BQKSyytxWB3SjjsGgK0lHsSMLiVZIKUrGNXwq5l1i7UNlTHr2wL3ADRozZFUrGyp2F9zy0vVoKkw89-RcQvfS-6RCKpWBUw1aEOnlaMpBO_cCixhqxF36vqnvnMcamHMkKARcxkgZDpxmtJVuOadNVlJ1O4-MQbR0msnurHQm3_vYm6azBsJbdDvQ0pAX6d68g-RaIYiCpxo1B5pAWEowdvKkVB6f_E_XQysHzbBBL511gg5yRHjTKyLaKeCvS6tI6Ue7T6kIGhTFkwxUjO7A6wjGPoc3esv4Ywli9A1S58DrPKbMV1rnq0dYBhQs59axT&sjdu=iMYN2Jly9sWyarqkEfXE5mk08BBOpcp3pYGzYADa6uVn7OC3DCWNiGDD5b1e2m2u2WJi_oaZ5ZeqGRnSSb1E0uTXjwX_1CW8-NhEEroXrMeyGmouCVyCauSCDVOG2g2tngKJn31rruz7NF1v4wPNSb5LDUCVObz6EAUsFekLErIpBJHaDsBLoJzh_X__sIWVI-zDlgpgF3wIMZanaHf28rApfn_Qx_WEgvZLmH_D2TfpAurgmrngMbUBeJA-VI12e7HFlosxcVUscCQmgoXPA6-1OCKfPzeoI9R1iY6aOfi2B4Nzs6CFo8N6ZuPsASj1eRKixf1VQqLQGo0CxEtWv73qCrFIBo3AuEc4mnCaZJLhHSCGQDbLZHnmCj0dRexGT3bfqTLONiZpZKzT7a41yZsY082_DSpTg9pIROgaL7dRSYbdN8ehgsYWosxq5_m-XowTNQFf7suJKsclP_UAygyLQzMytTZ1ja1hv9PiAOmy_LtdzpowrbxNaHvChHf-PoNUDV7KPMvt6bY4SA84nJRzlM013kjyswnNEDy-sB8zSDlgrtbbv8t7Uy9Ec8jsyG9gXKIdqZmHXu1xJZqn1A29iiZgUGDloMInzfiZRCQ')
data = driver.find_element(By.CLASS_NAME, "icl-u-xs-mt--xs.icl-u-textColor--secondary.jobsearch-JobInfoHeader-subtitle.jobsearch-DesktopStickyContainer-subtitle")
print(data.text)

In [ ]:
x = data.text.split('\n')
print(x[0])

In [ ]:
benefits = driver.find_elements(By.CLASS_NAME, "css-tvvxwd.ecydgvn1")
print(benefits[3].text)

In [ ]:
job_activity = driver.find_element(By.CLASS_NAME, "jobsearch-HiringInsights-entry--text")
print(job_activity.text)

In [ ]:
job_title, company_name, company_location, job_description, job_activity, benefits_list = scrape_from_individual_job_url("")

In [ ]:
job_title

In [ ]:
company_name

In [ ]:
company_location

In [ ]:
job_description

In [ ]:
job_activity

In [ ]:
benefits_list

In [ ]:
# Copied from this stackoverflow link. Maybe can fix? https://stackoverflow.com/questions/51775122/nosuchwindowexception-no-such-window-window-was-already-closed-while-switchi

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

url = "your_url"
driver = webdriver.Chrome(service=ser, options=options)
driver.get(url)
windows_before  = driver.current_window_handle
driver.find_element_by_id('portal.scheduling.prepopulate').click()
WebDriverWait(driver, 5).until(EC.number_of_windows_to_be(2))
windows_after = driver.window_handles
new_window = [x for x in windows_after if x != windows_before][0]
# driver.switch_to_window(new_window) <!---deprecated>
driver.switch_to.window(new_window)
WebDriverWait(driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.ID,"myframe"))) # or By.NAME, By.XPATH, By.CSS_SELECTOR

In [ ]:
df = scrape_all_job_data("Data Scientist", "Philadelphia")

In [ ]:
df.head()